In [1]:
import os
import sys
sys.dont_write_bytecode = True
os.environ["NUMBA_DISABLE_JIT"] = "1"

import pandas as pd
import vectorbt as vbt
import numpy as np
from quantfreedom.backtester.base.base import rsi_below_entries, plot_trades_all_info
from quantfreedom.backtester.nb.simulate import simulate_from_signals
from quantfreedom.backtester.enums.enums import (
    LeverageMode,
    SizeType,
    OrderType,
    SL_BE_or_Trail_BasedOn,
)
from quantfreedom.utils.caching import clear_cache

pd.options.display.float_format = '{:,.2f}'.format

prices = pd.read_csv('30min.csv', index_col='time')

vbt.IndicatorFactory.from_talib

entries = rsi_below_entries(
    timeperiods=[15],
    below_ranges=[40],
    prices=prices.close
)


In [3]:
final_array, order_records = simulate_from_signals(
    open_prices=prices.open.values,
    high_prices=prices.high.values,
    low_prices=prices.low.values,
    close_prices=prices.close.values,
    entries=entries.values,
    equity=1000.,
    fee_pct=.06,
    mmr=.5,
    lev_mode=LeverageMode.LeastFreeCashUsed,
    size_type=SizeType.RiskPercentOfAccount,
    order_type=OrderType.LongEntry,
    max_equity_risk_pct=4,
    risk_rewards=[3,5,6],
    size_pct=1.,
    gains_pct_filter=40,
    sl_pcts=[2,3,4],
    # tsl_true_or_false=True,
    # tsl_pcts_init=np.arange(2,5,1),
    # tsl_based_on=SL_BE_or_Trail_BasedOn.low_price,
    # tsl_trail_by_pct=np.arange(1,4,1),
    # tsl_when_pct_from_avg_entry=np.arange(1,4,1),
)


In [9]:
order_records.shape

(13920,)

In [10]:
df_or = pd.DataFrame(order_records)
for i in range(len(OrderType._fields)):
    df_or.replace({'order_type': {i: OrderType._fields[i]}}, inplace=True)
df_or.head(20)    

,order_id,settings_id,bar,size_value,price,avg_entry,fees_paid,order_type,real_pnl,equity,sl_prices,tsl_prices,tp_prices
0,0,0,31,471.97,"10,850.00","10,850.00",NaN,LongEntry,NaN,"1,000.00","10,633.00",NaN,"11,553.11"
1,1,0,32,471.97,"10,725.00","10,787.14",NaN,LongEntry,NaN,"1,000.00","10,571.40",NaN,"11,486.18"
2,2,0,32,943.93,"10,571.40","10,787.14",1.12,LongSL,-20.00,980.00,"10,571.40",NaN,"11,486.18"
3,3,0,33,462.53,"10,577.50","10,577.50",NaN,LongEntry,NaN,980.00,"10,365.95",NaN,"11,262.95"
4,4,0,34,462.53,"10,523.00","10,550.18",NaN,LongEntry,NaN,980.00,"10,339.18",NaN,"11,233.86"
5,5,0,35,462.53,"10,516.00","10,538.76",NaN,LongEntry,NaN,980.00,"10,327.99",NaN,"11,221.70"
6,6,0,36,462.53,"10,489.00","10,526.28",NaN,LongEntry,NaN,980.00,"10,315.75",NaN,"11,208.41"
7,7,0,436,"1,850.10","11,208.41","10,526.28",2.29,LongTP,117.60,"1,097.60","10,315.75",NaN,"11,208.41"
8,8,0,522,518.03,"11,338.00","11,338.00",NaN,LongEntry,NaN,"1,097.60","11,111.24",NaN,"12,072.74"
9,9,0,549,518.03,"11,349.00","11,343.50",NaN,LongEntry,NaN,"1,097.60","11,116.63",NaN,"12,078.59"


In [8]:
df_cart = pd.DataFrame(final_array).dropna(axis='columns', thresh=1)
for i in range(len(SL_BE_or_Trail_BasedOn._fields)):
    df_cart.replace({'tsl_based_on': {i: SL_BE_or_Trail_BasedOn._fields[i]}}, inplace=True)
    df_cart.replace({'sl_to_be_based_on': {i: SL_BE_or_Trail_BasedOn._fields[i]}}, inplace=True)
df_cart.T

,0,1,2
total_trades,150.00,54.00,32.00
gains_pct,43.67,202.62,300.07
win_rate,25.33,24.07,31.25
to_the_upside,0.23,0.25,0.59
total_pnl,436.73,"2,026.16","3,000.74"
ending_eq,"1,436.73","3,026.16","4,000.74"
settings_id,0.00,1.00,2.00
max_equity_risk_pct,4.00,4.00,4.00
risk_rewards,3.00,5.00,6.00
size_pct,1.00,1.00,1.00


In [ ]:
plot_trades_all_info(
    open_prices=prices.open,
    high_prices=prices.high,
    low_prices=prices.low,
    close_prices=prices.close,
    order_records=order_records[:30],
)

In [ ]:
clear_cache()